# RAG avec Mistral (Retrieval-Augmented Generation)
Fait par les Etudiants du Matser 2 :
- Mohamed DIALLO
- Yaya SANAOGO

**FST-USTTB**

Ce système est conçu pour extraire des informations pertinentes d'un document PDF et générer des réponses à des questions basées sur ces informations.

## Import des packages

In [1]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
import os
import pickle

## Copie du model Mistral 
1. **Chargement de la copie mistral en local (Ollama) :**

Pour des certaines raisons nous avons opter pour faire une copie du model mistral en local (ModelFile) a quelle comme temperature du 1 pour cette copie

*sources* : 
- [Running models with Ollama step-by-step](https://medium.com/@gabrielrodewald/running-models-with-ollama-step-by-step-60b6f6125807) 

- [ollama_custom_model](https://unmesh.dev/post/ollama_custom_model/)

- [How to Customize LLMs with Ollama](https://medium.com/@sumudithalanz/unlocking-the-power-of-large-language-models-a-guide-to-customization-with-ollama-6c0da1e756d9)

In [2]:
# Initialize model mistral
model = Ollama(model="mistral_copy")

## Chargement et Prétraitement des Documents

2. **Chargement du Document PDF avec PyMuPDFLoader:**

Il s'agit de l'option d'analyse PDF la plus rapide. Elle contient des métadonnées détaillées sur le PDF et ses pages, et renvoie un document par page.

source : [Using PyMuPDF](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/#using-pymupdf)

Le document PDF est chargé à l'aide de PyMuPDFLoader depuis le chemin : /home/mohamed/Documents/Mohamed/2022-13_compressed.pdf.

In [3]:
pdf_path = "/home/mohamed/Downloads/2022-13_compressed.pdf"
loader = PyMuPDFLoader(pdf_path)
doc = loader.load()

In [4]:
# Questions sans RAG
question_1 = "Quelles sont les politiques économiques les plus récentes mises en place par le gouvernement malien en 2024 ?"
question_2 = "Pouvez-vous fournir des détails sur les réformes éducatives les plus récentes au Mali ?"

In [6]:
response_1 = model.invoke(question_1)

print(f"Question: \n{question_1}")
print(f"Réponse du LLM seul:\n{response_1}\n")

Question: 
Quelles sont les politiques économiques les plus récentes mises en place par le gouvernement malien en 2024 ?
Réponse du LLM seul: 
 Je n'ai pas accès à des informations actuelles en temps réel. Il est donc impossible pour moi de vous dire quelles sont les politiques économiques les plus récentes mises en place par le gouvernement malien en 2024, car je suis un modèle de texte basé sur l'IA et j'ai été créé en 2021. Pour obtenir des informations actuelles, je recommande de consulter le site internet du gouvernement malien ou d'autres sources fiables d'informations économiques telles que les médias ou des ressources Internet comme la Banque Mondiale ou l'ONU.



In [8]:
response_2 = model.invoke(question_2)

print(f"Question: \n{question_2}")
print(f"Réponse du LLM seul:\n{response_2}\n")

Question: 
Pouvez-vous fournir des détails sur les réformes éducatives les plus récentes au Mali ?
Réponse du LLM seul:
 Bien sûr ! Les réformes éducatives ont été une priorité importante pour le gouvernement du Mali pendant plusieurs années. Voici quelques points clés de ces réformes :

1. Rénovation de l'Éducation Primaire et de l'Alphabétisation Générale : Cette réforme, engagée en 2015, vise à garantir une qualité éducative accrue en primaire, tout en améliorant l'alphabétisation générale dans le pays. Les changements incluent une augmentation du nombre d'heures d'apprentissage, la mise en place de programmes pédagogiques et d'outils de gestion à échelle nationale, ainsi qu'un accent mis sur l'évaluation régulière et adaptative des élèves.

2. Rénovation du Système d'Enseignement Supérieur : Cette réforme, engagée en 2016, vise à améliorer la qualité de l'enseignement supérieur au Mali. Les changements incluent un processus plus rigoureux de sélection des étudiants, un système d'év

Ces questions nécessitent des informations très récentes que le modèle de langage seul (LLM) ne peut pas connaître sans l'aide de documents externes récents.
La preuve avec les reponses des deux questions l'une qui dit clairement et l'autre qui vas repondre avec des donnees basé sur des annees 2015 à 2018

3. **Découpage du Texte :**

Pour faciliter l'indexation et la recherche, le texte du PDF est découpé en morceaux plus petits. Ce découpage est réalisé avec RecursiveCharacterTextSplitter, utilisant des séparateurs variés (sauts de ligne, espaces, ponctuations). Chaque morceau de texte a une taille maximale de 300 caractères avec un chevauchement de 100 caractères pour conserver le contexte entre les morceaux.

LangChain propose différents types de séparateurs de texte suivant le tableau recapitulatif ci-dessous (premier lien dans le block des sources)
| Nom                               | Classes                                      | Sépare sur                     | Ajoute des métadonnées | Description                                                                                                                                                                                                                       |
|-----------------------------------|----------------------------------------------|-------------------------------|------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Récursif                          | RecursiveCharacterTextSplitter, RecursiveJsonSplitter | Une liste de caractères définis par l'utilisateur |                        | Sépare le texte de manière récursive. Cette méthode essaye de garder les morceaux de texte liés les uns aux autres. C'est la méthode recommandée pour commencer à séparer du texte.                                                |
| HTML                              | HTMLHeaderTextSplitter, HTMLSectionSplitter  | Caractères spécifiques à HTML | ✅                     | Sépare le texte en fonction des caractères spécifiques à HTML. Notamment, cela ajoute des informations pertinentes sur l'origine de ce morceau (basé sur le HTML).                                                                |
| Markdown                          | MarkdownHeaderTextSplitter                   | Caractères spécifiques à Markdown | ✅                     | Sépare le texte en fonction des caractères spécifiques à Markdown. Notamment, cela ajoute des informations pertinentes sur l'origine de ce morceau (basé sur le Markdown).                                                        |
| Code                              | plusieurs langages                           | Caractères spécifiques au code (Python, JS) |                        | Sépare le texte en fonction des caractères spécifiques aux langages de programmation. 15 langues différentes sont disponibles au choix.                                                                                          |
| Token                             | plusieurs classes                            | Tokens                        |                        | Sépare le texte en tokens. Il existe plusieurs manières de mesurer les tokens.                                                                                                                                                    |
| Caractère                         | CharacterTextSplitter                        | Un caractère défini par l'utilisateur |                        | Sépare le texte en fonction d'un caractère défini par l'utilisateur. C'est l'une des méthodes les plus simples.                                                                                                                   |
| [Expérimental] Segmenteur Sémantique | SemanticChunker                             | Phrases                       |                        | Sépare d'abord en phrases. Puis combine celles qui sont adjacentes si elles sont suffisamment similaires sémantiquement. Tiré de Greg Kamradt.                                                                                     |
| AI21 Segmenteur de Texte Sémantique | AI21SemanticTextSplitter                    |                             | ✅                       | Identifie des sujets distincts qui forment des morceaux de texte cohérents et les sépare en conséquence.                                                                                                                           |


sources :

- [Types of Text Splitters](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/#types-of-text-splitters) 

- [Splitting text from languages without word boundaries](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries)

In [9]:
# Définir les séparateurs pour le découpage du texte
separators = [
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
]

In [13]:
# Initialiser le découpeur de texte avec les paramètres spécifiés
text_splitter = RecursiveCharacterTextSplitter(
    separators=separators,
    chunk_size=300, # Taille de chaque chunk en caractères
    chunk_overlap=100, # Chevauchement entre les chunks consécutifs
    length_function=len, # Fonction pour calculer la longueur du texte
    add_start_index=True, # Ajouter l'index de début à chaque chunk
)

In [14]:

chunks = text_splitter.split_documents(doc)
print(f"Split {len(doc)} documents into {len(chunks)} chunks.")

Split 121 documents into 583 chunks.


In [15]:
# Afficher un exemple de contenu de page et de métadonnées pour un chunk
page = chunks[0]
print(page.page_content)
print(page.metadata)

Enquête d’opinion «Que pensent les Malien(ne)s ?»
1
Chèr(e)s ami(e)s de la Friedrich-Ebert-Stiftung 
au Mali,  
 
Depuis 10 ans, nous demandons "Qu'en 
pensent les Malien(ne)s ? Lorsque nous avons 
commencé le Mali-Mètre en 2012, nous 
n'avions pas conscience de la portée qu'il
{'source': '/home/mohamed/Downloads/2022-13_compressed.pdf', 'file_path': '/home/mohamed/Downloads/2022-13_compressed.pdf', 'page': 2, 'total_pages': 121, 'format': 'PDF 1.4', 'title': 'Mise en page 1', 'author': 'IMPRIM COLOR', 'subject': '', 'keywords': '', 'creator': 'QuarkXPress(R) 14.21', 'producer': 'iLovePDF', 'creationDate': 'D:20220512192754Z', 'modDate': 'D:20240726083929Z', 'trapped': '', 'start_index': 0}


## Création et Gestion des Embeddings

4. **Génération des Embeddings :**

Les embeddings, qui sont des représentations vectorielles des morceaux de texte, sont générés à l'aide du modèle sentence-transformers/all-MiniLM-L6-v2 fourni par HuggingFaceEmbeddings.

Pourquoi le modèle sentence-transformers/all-MiniLM-L6-v2 alors qu'on a passé beaucoup de temps à télécharger Mistral et là, on vient nous parler d'un autre modèle voilà la question que l'on c'était poser au tout debut mais en des relectures et quelque recherche on as su que le sentence-transformers c'était juste pour nous aider dans la construction de notre base vectorielle il y en plein d'autre le plus connu reste le all-MiniLM-l6-V2 enfin disons par défaut. 

*sources :*

- [Embedding models](https://ollama.com/blog/embedding-models)

- [Using Langchain, Chroma, and GPT for document-based retrieval-augmented generation](https://developer.dataiku.com/12/tutorials/machine-learning/genai/nlp/gpt-lc-chroma-rag/index.html)

In [16]:
# Récupérer la fonction d'embeddings à partir des ressources du code env
emb_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=emb_model)

/home/mohamed/envpy/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/mohamed/envpy/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2024-07-30 20:43:48.356197: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the e

5. **Cache des Embeddings :**

Pour optimiser les performances, les embeddings sont soit chargés depuis un cache (emb_cache.pkl), soit générés et ensuite enregistrés pour une utilisation future si le cache n'existe pas.

In [17]:
# Chemin vers le répertoire pour enregistrer la base de données Chroma
CHROMA_PATH = "embdb"
EMB_CACHE_PATH = "emb_cache.pkl"

In [18]:
# Charger les embeddings depuis le cache ou les générer si le cache n'existe pas
if os.path.exists(EMB_CACHE_PATH):
    with open(EMB_CACHE_PATH, 'rb') as f:
        cached_embeddings = pickle.load(f)
else:
    # Générer les embeddings pour chaque chunk
    cached_embeddings = embeddings.embed_documents([chunk.page_content for chunk in chunks])
    with open(EMB_CACHE_PATH, 'wb') as f:
        pickle.dump(cached_embeddings, f)

## Indexation et Recherche

6. **Base de Données Chroma :**

Les morceaux de texte, accompagnés de leurs embeddings, sont stockés dans une base de données Chroma. 
Cette base de données est persistée dans le répertoire embdb pour assurer une sauvegarde sur le disque.

In [19]:
# Créer une nouvelle base de données Chroma et ajouter les documents
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
db.add_documents(documents=chunks)

/home/mohamed/envpy/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


['0a45d8e4-3728-4809-a546-6264f2553f86',
 'c68c5cd1-edfe-4ad9-9e38-4f300922b20f',
 '11a8bb04-1d60-467c-8cd5-5f952e366806',
 '5eace7d8-672f-4eed-97e4-0f632d13351f',
 'cf7319b3-2def-48b6-89e7-2521ed4b681d',
 '9066f8a6-4cea-4cdb-9f9b-557c6d66fdb1',
 'b2833db3-1abe-4d18-ba4b-db3a787e073e',
 'cc59aeca-32e8-48d8-a925-2e51dadd0dc4',
 'b70e9c44-dfba-40f8-97b5-c486cd875494',
 '001f648e-f625-4b7f-8ec1-ca04bad67f07',
 '95bb657c-403b-4ba0-87f9-b71d6a795940',
 '0a00a015-ca92-4856-ab9f-c07288841897',
 '6fdaffff-ad79-4288-82bf-2138887e388b',
 '6b90d2d2-e07e-4e51-b41a-5e2d97a2d505',
 '76d15e9a-508b-4501-980b-2174a22ba6d2',
 'aafd0128-d4f9-495d-8f9e-b74feacf218f',
 '1d551f43-6add-449c-999f-886a4ce453e3',
 'daa73992-12d0-4f3f-a217-ea346131e2dd',
 '6a228bee-2201-427d-9189-498f507c55f9',
 '50c68bfc-0077-4d01-9edc-54f5df7f8fdc',
 '2f5271e4-17c5-48c1-99a2-1dc6e636e072',
 '0e0b88b1-b9af-4a94-84e9-ed2653375cb3',
 '3ae33331-e588-487d-a725-11415a1d67e9',
 'b6d8f9f1-4ced-4ac0-ab25-1d0fc58e267b',
 '3fd3614a-1ac1-

In [20]:
# Persister la base de données sur le disque
db.persist()
print(f"Enregistrement de {len(chunks)} chunks dans {CHROMA_PATH}.")

Enregistrement de 583 chunks dans embdb.


/home/mohamed/envpy/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


### Première question :

In [21]:
query_text = question_1

7. **Recherche par Similarité et Ranking:**

Lorsqu'une question est posée, le système effectue une recherche par similarité dans la base de données Chroma.
Les documents les plus pertinents sont récupérés en fonction de leur similarité avec la question avec mecanisque de ranking.

In [22]:
# Retrieve context from DB using similarity search
results = db.similarity_search_with_relevance_scores(query_text, k=3)  # Récupérer plus de résultats pour le ranking
results.sort(key=lambda x: x[1], reverse=True)  # Tri par score de pertinence
top_results = results[:3]  # Prendre les 3 meilleurs résultats

context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in top_results])

## Génération de Réponses avec Compression du Prompt

8. **Modèle de Prompt :**

Un modèle de prompt est défini pour structurer la réponse. Le prompt intègre le contexte extrait des documents pertinents pour guider le modèle Mistral dans la génération de réponses claires et utiles.

9. **Compression du Prompt :**

Avant d'intégrer la question dans le modèle de prompt, une phase de compression du prompt est réalisée. 
Cela implique de réduire la longueur du contexte tout en conservant les informations essentielles. 
Cette étape permet de rester dans les limites de longueur du modèle tout en maintenant la pertinence de la réponse.

In [23]:
# Function to compress prompts
def compress_prompt(context, question):
    # Keep only the most relevant parts of the context
    compressed_context = context[:1000]  # Limiter à 1000 caractères par exemple
    return f"Question: {question}\nContext: {compressed_context}"

In [27]:
# Compress the prompt
compressed_prompt = compress_prompt(context_text, query_text)

In [31]:
# Create prompt using compressed context and query text
prompt_template = ChatPromptTemplate.from_template("""
    Tu t'appelles Okka.
    Tu es un assistant en intelligence artificielle conçu pour aider les utilisateurs en récupérant des informations pertinentes et en générant des réponses basées sur ces informations.
    Ton objectif est de fournir des réponses claires et utiles.
    Repond toujours dans la même langue que la question

    {compressed_prompt}
    """)

In [24]:
# Create prompt using compressed context and query text
prompt_template = ChatPromptTemplate.from_template(
    """
        Tu t'appelles Okka.
        Tu es un assistant en intelligence artificielle conçu pour aider les utilisateurs en récupérant des informations pertinentes et en générant des réponses basées sur ces informations. 
        Ton objectif est de fournir des réponses claires et utiles.

        L'utilisateur a posé la question suivante : "{question}"
        Recherche des documents pertinents dans la base de connaissances et utilise ces informations pour répondre à la question.

        Réponds à la question en utilisant uniquement le contexte suivant :
        {context}
        - -
        Réponds à la question ci-dessous en te basant uniquement sur le contexte fourni, dans la même langue que la question :
        Question : {question}
    """
)

10. **Génération de Réponse :**

La question posée est intégrée dans le modèle de prompt avec le contexte compressé, et la réponse est générée par le modèle Mistral.

In [26]:
# Generate response based on the prompt
prompt = prompt_template.format(context=context_text, question=query_text)

response_text = model.invoke(prompt_template.format(context=context_text, question=query_text))
#response_text = model.invoke(prompt_template.format(compressed_prompt=compressed_prompt))

# Get sources of the relevant documents
sources = [doc.metadata.get("source", None) for doc, _score in top_results]

# Format and return the response
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response:  Bien sûr, il n'y a pas de politique économique précise citée pour l'année 2024 dans le contexte fourni. Les principales thématiques abordées pour les années 2017-2022 sont la transition politique, les défis et priorités du Mali, ainsi que des enquêtes d'opinion sur la confiance de la population dans cette transition. Il est donc difficile de fournir une réponse précise à votre question à partir des informations disponibles.
Sources: ['/home/mohamed/Downloads/2022-13_compressed.pdf', '/home/mohamed/Downloads/2022-13_compressed.pdf', '/home/mohamed/Downloads/2022-13_compressed.pdf']


### Posons une nouvelle question :

In [27]:
query_text = question_2
response_text = model.invoke(prompt_template.format(context=context_text, question=query_text))
# Get sources of the relevant documents
sources = [doc.metadata.get("source", None) for doc, _score in top_results]

# Format and return the response
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response:  Je suis désolé, mais mon contexte de référence ne mentionne pas explicitement de réforme éducative récente spécifique durant la période 2017-2022. Toutefois, il existe une enquête d'opinion qui montre que le renforcement de l'éducation est une priorité majeure des Malien(ne)s. En ce qui concerne les réformes éducatives au Mali au cours de cette période, je peux seulement vous fournir quelques informations générales en provenance de sources externes.

Dans le cadre du programme de développement national du Mali (PNDM) 2017-2021, la priorité principale concernant l'éducation était d'améliorer les résultats scolaires et de réduire les inégalités géographiques dans l'accès à l'éducation. Afin de remplir cette priorité, le gouvernement malien a mis en place des mesures telles que :

1. La mise en œuvre du programme national d’apprentissage et de performance (PNAP), qui vise à améliorer les résultats scolaires à tous les niveaux de l'éducation.
2. L'augmentation du budget alloué à

In [28]:
query_text = "Quelles sont les principales réformes engagées et dans quelles domaines ?"
response_text = model.invoke(prompt_template.format(context=context_text, question=query_text))
# Get sources of the relevant documents
sources = [doc.metadata.get("source", None) for doc, _score in top_results]

# Format and return the response
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response:  Les principales réformes engagées au Mali pour les cinq années suivantes (2017-2022) portent principalement sur trois domaines : la paix et sécurité, l'emploi pour les jeunes et la sécurité alimentaire, ainsi que la relance de l'économie. La majorité des Maliens espèrent voir un renforcement de la bonne gouvernance avec cette transition politique en cours.
Sources: ['/home/mohamed/Downloads/2022-13_compressed.pdf', '/home/mohamed/Downloads/2022-13_compressed.pdf', '/home/mohamed/Downloads/2022-13_compressed.pdf']


# Evaluation des réponses du RAG avec le BERTScore 

Bertscore est une mesure qui est apparue comme une alternative aux mesures d'évaluation traditionnelles dans le domaine du traitement du langage naturel (NLP). Elle est particulièrement utile pour évaluer la qualité du résumé de texte, en mesurant la similitude du résumé de texte avec le texte d'origine

source : 

[BERTScore expliqué en 5 minutes](https://medium.com/@abonia/bertscore-explained-in-5-minutes-0b98553bfb71)

Ce article explique la motivation derrière BERTScore et détaille son architecture dont l'illustration est cidessous en image aussi
l'artilce explique les avantages et incovenients de l'utilisation de Bert score.
![Architecture](https://github.com/Mohameddiallo728/chatLLM/blob/main/Diagram-of-BERTScore-Retrieved-from-53.jpg)

# Résultats

Le système RAG développé permet de traiter efficacement un document PDF et d'extraire des informations pertinentes pour répondre à des questions spécifiques. La réponse générée inclut également les sources des documents pour assurer la traçabilité des informations fournies. La phase de compression du prompt permet d'optimiser la pertinence et la concision des réponses.